<a href="https://colab.research.google.com/github/anacasicande/Modelamiento/blob/main/Tarea2_Ana.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import sympy as sp
#Primero importamos la librería que se va a utilizar, en este caso Numpy que será
# utilizada para facilitar el uso de ciertas ecuaciones.
#Luego definimos las variables que serán utilizadas a lo largo de todo el código

# Constantes
R = 0.08314  # bar·L/(mol·K), constante de los gases
T = 350  # K, temperatura
P = 9.4573  # bar, presión de vapor saturado
T_c = 425.1  # K, temperatura crítica del n-butano
P_c = 37.96  # bar, presión crítica del n-butano

T_r= T/T_c #Definimos la ecuación de la temperatura reducida
P_r=P/P_c #Definimos la ecuación de la presión reducica
Psi= 0.42748 #Este valor se saca de la tabla 3.1 en la fila de RK para la columna de PSI
Omega= 0.08664 #Este valor se encuentra en la tabla 3.1 en la fila de RK para la columna de OMEGA
Z=sp.Symbol('Z') #Utilizamos el Symbol de la librería Sympy para tratar la Z como un valor

# Definimos los valores que serán utilizados para la ecuación de Redlich-Kwong
a =(Psi * (T_r**(-1/2))/(Omega*T_r)) #Hacemos el cálculo para encontrar el valor de la q
b = Omega *(P_r/T_r) #Hacemos el cálculo de lo que será nuestra Betha

# Defino la función de Redlich-Kwong para vapor
def f_VapRK(Z):
    H=Z-(1+b-(b*a)*((Z-b)/(Z*(Z+b)))) # Igualo la ecuación para vapor segur RK,
    # igualo la ecuación a una variable y le resto el término al que estaría igualado Z
    return H #Me devuelve la variable

#Se hace la derivada de la funcion que encuentra los valores de z en Redlich-Kwong
def f_dVapRK(Z_value): #
  devvapor= sp.diff(f_VapRK(Z),Z) #Abrimos la librería sympy la cual es la
  # encargada de derivar la función para el vapor con respecto a Z
  M=devvapor.subs(Z,Z_value) #Aquí obtenemos el valor de la derivada simbólica que definimos
  # anteriormente y esto nos permite trabajar con Z no como un número, lo que permite
  # derivar con respecto a ella
  return M.evalf() # Convierte el valor simbólico con el que estábamos trabajando
  # en un valor numérico

# Defino la ecuación Redlich-Kwong para líquido
def f_LiqRK(Z):
    T=Z-(b+(Z*(Z+b)*((1+b-Z)/(a*b)))) #Hacemos lo mismo que hicimos para el vapor
    #pero esta vez para el líquido despejando la ecuación de líquido la cual estamos
    #igualando a la variable que esta vez sería T
    return T

#Defino la función que va a derivar la ecuación de RK para el líquido
def f_dLiqRK(Z_value):
  devliq= sp.diff(f_LiqRK(Z),Z) #Abrimos la librería sympy la cual es la
  # encargada de derivar la función para el líquido con respecto a Z
  N=devliq.subs(Z,Z_value)#Aquí obtenemos el valor de la derivada simbólica que definimos
  # anteriormente y esto nos permite trabajar con Z no como un número, lo que permite
  # derivar con respecto a ella
  return N.evalf() # Convierte el valor simbólico con el que estábamos trabajando
  # en un valor numérico

# Método de Newton-Raphson
def newton_raphson(funV,funP,Z, tol=1e-10, max_iter=1000): #Definimos la función
# de Newton-Raphson la cual recube dos funciones, la del vapor y la del líquido,
# recibe el valor que ya se había tomado de Z
    x = Z
    for i in range(max_iter):
        fx = funV(x)
        fpx = funP(x)

        fpx_val= float(fpx)

        if abs(fpx) < tol:
            print("La derivada es muy pequeña, no se puede continuar")
            return None

        # Fórmula de Newton-Raphson
        z_new = x - fx / fpx_val

        # Verificar la convergencia
        if abs(z_new - x) < tol:
            print(f"Converge en la iteración {i+1}")
            return z_new

        x = z_new

    print("No converge después del número máximo de iteraciones")
    return None

# Valores iniciales para la fase vapor y la fase líquida
Z_inicialvap = 1  # Volumen inicial de vapor saturado (en L/mol)
Z_inicialliq = 0.005  # Volumen inicial de líquido saturado (en L/mol)

# Llamar al método de Newton-Raphson para vapor saturado
V_vapor = newton_raphson(f_VapRK, f_dVapRK, Z_inicialvap)*1000*R*T/P #Además de
# hacer el método de Newton-Raphson, se termina multiplica la Z por 1000 ya que antes estaba
# en valores de litros, ahora está en centímetros cúbicos. Y también se mmultiplica
# por la constante de los gases ideales R y por la temperatura, se divide por
# la presión y con esto podemos hayar el volúmen del vapor

if V_vapor is not None:
    print(f"El volumen molar del vapor saturado es: {V_vapor:.2f} cm^3/mol")#Imprimimos
    # el valor que se obtiene para el vapor
    print(f"La Z para este caso es: {(V_vapor*P/(1000*R*T)):.3f}")# Imprimimos
    # el valor de la Z mediante la multiplicación de la presión sobre 1000, la constante
    # de gases ideales y la temperatura.


# Llamar al método de Newton-Raphson para líquido saturado
V_liquid = newton_raphson(f_LiqRK, f_dLiqRK, Z_inicialliq)*1000*R*T/P#Además de
# hacer el método de Newton-Raphson, se termina multiplica la Z por 1000 ya que antes estaba
# en valores de litros, ahora está en centímetros cúbicos. Y también se mmultiplica
# por la constante de los gases ideales R y por la temperatura, se divide por
# la presión y con esto podemos hayar el volúmen del líquido

if V_liquid is not None:
    print(f"El volumen molar del líquido saturado es: {V_liquid:.2f} cm^3/mol")#Imprimimos
    # el valor que se obtiene para el volumen del líquido
    print(f"La Z para este caso es: {(V_liquid*P/(1000*R*T)):.3f}")# Imprimimos
    # el valor de la Z mediante la multiplicación de la presión sobre 1000, la constante
    # de gases ideales y la temperatura.


Converge en la iteración 4
El volumen molar del vapor saturado es: 2555.32 cm^3/mol
La Z para este caso es: 0.830
Converge en la iteración 6
El volumen molar del líquido saturado es: 133.27 cm^3/mol
La Z para este caso es: 0.043
